# Aligns each ICESat-2 ATL06 data to the grid set by flow-data

* Output file contaning data is normal filename with a `.csv` extension

In [1]:
import ICESat2GroundingLineMigration.IceSatHDF5Unpacker as unpack
import ICESat2GroundingLineMigration.GLineUnpacker as gline
import ICESat2GroundingLineMigration.FlowUnpacker as flow
import ICESat2GroundingLineMigration.Visualizations as visualize
import numpy as np
import csv

Import the altimetry data and flow data

In [2]:
dataset = unpack.Dataset("ATL06") # grab altimetry data

filenames = dataset.datafiles # datafiles is a list of all .h5 files in the directory

flowdatabase = flow.Database("Flow/antarctic_ice_vel_phase_map_v01.nc") # grab flow dataset

### Compute all of the flow data parameters
Produces the following: *Angle, Speed, Error, Angle Error*

From the following: *Error vector, Latitude, Longitude, x, y*

**This requires more than 8GB of RAM**

In [3]:
flowdatabase.compute_all()

### Iterate through the files and sort onto the xy-flow data grid

In [4]:

for file in filenames:
    print(f"Sorting file {file}")
    granule = dataset.openfilename(file)
    
    laserdata = []
    for laser in granule.lasers:
        print(f"On laser {laser.beamnum}/6")
        
        lat, lon, time, dh_fit_dx, dh_fit_dx_sigma, metadata, granuledata = laser.getTrackData()
        xyindices, trackx, tracky = unpack.Basemap.latlonindex_to_grid(lat, lon, flowdatabase)
        
        laserdata.append([xyindices, trackx, tracky])
        
    maxlength = 0
    for data in laserdata:
        if len(data[0]) > maxlength:
            maxlength = len(data[0])
            
    for data in laserdata:
        if len(data[0]) < maxlength:
            difference = maxlength - len(data[0])
            data[0], data[1], data[2] = np.append(data[0], [None]*difference), np.append(data[1], [None]*difference), np.append(data[2], [None]*difference)
        
    print("Writing File")
    with open(f'ATL06/{file.split(".")[0]}.csv', 'w+') as outfile:
        filewrite = csv.writer(outfile, delimiter=',')
        filewrite.writerow(["index", "l1-index", "l1-x", "l1-y", "r1-index", "r1-x", "r1-y", "l2-index", "l2-x", "l2-y", 
                            "r2-index", "r2-x", "r2-y", "l3-index", "l3-x", "l3-y", "r3-index", "r3-x", "r3-y"])
        for i in range(maxlength):
            filewrite.writerow([i, laserdata[0][0][i], laserdata[0][1][i], laserdata[0][2][i], 
                                laserdata[1][0][i], laserdata[1][1][i], laserdata[1][2][i], 
                                laserdata[2][0][i], laserdata[2][1][i], laserdata[2][2][i],
                                laserdata[3][0][i], laserdata[3][1][i], laserdata[3][2][i], 
                                laserdata[4][0][i], laserdata[4][1][i], laserdata[4][2][i], 
                                laserdata[5][0][i], laserdata[5][1][i], laserdata[5][2][i]])

Sorting file ATL06_20221012195854_03371710_005_01.h5
On laser 1/6
On laser 2/6
On laser 3/6
On laser 4/6
On laser 5/6
On laser 6/6
Writing File
Sorting file ATL06_20221012214635_03381712_005_01.h5
On laser 6/6
On laser 5/6
On laser 4/6
On laser 3/6
On laser 2/6
On laser 1/6
Writing File
Sorting file ATL06_20200202050614_05720611_005_01.h5
On laser 6/6
On laser 5/6
On laser 4/6
On laser 3/6
On laser 2/6
On laser 1/6
Writing File
Sorting file ATL06_20200201053154_05570611_005_01.h5
On laser 6/6
On laser 5/6
On laser 4/6
On laser 3/6
On laser 2/6
On laser 1/6
Writing File
Sorting file ATL06_20200203184911_05960611_005_01.h5
On laser 6/6
On laser 5/6
On laser 4/6
On laser 3/6
On laser 2/6
On laser 1/6
Writing File
Sorting file ATL06_20200202045834_05720610_005_01.h5
On laser 1/6
On laser 2/6
On laser 3/6
On laser 4/6
On laser 5/6
On laser 6/6
Writing File
Sorting file ATL06_20200203185454_05960612_005_01.h5
On laser 6/6
On laser 5/6
On laser 4/6
On laser 3/6
On laser 2/6
On laser 1/6
Writi